<a href="https://colab.research.google.com/github/MostafaJahanian/Free_Projects/blob/main/chest_xray_classification_using_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download paultimothymooney/chest-xray-pneumonia

 99% 2.28G/2.29G [01:07<00:00, 17.8MB/s]
100% 2.29G/2.29G [01:07<00:00, 36.3MB/s]


In [ ]:
! unzip chest-xray-pneumonia.zip

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import random

In [2]:
tf.random.set_seed(42)

train_set = keras.utils.image_dataset_from_directory('chest_xray/train',
                                                     validation_split = 0.05,
                                                     subset = 'training',
                                                     seed = 1)

val_set_extra = keras.utils.image_dataset_from_directory('chest_xray/train',
                                                     validation_split = 0.05,
                                                     subset = 'validation',
                                                     seed = 1)

Found 5216 files belonging to 2 classes.
Using 4956 files for training.
Found 5216 files belonging to 2 classes.
Using 260 files for validation.


In [3]:
test_set = keras.utils.image_dataset_from_directory('chest_xray/test')
val_set_default = keras.utils.image_dataset_from_directory('chest_xray/val')

Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [4]:
val_set_default.class_names, val_set_extra.class_names

(['NORMAL', 'PNEUMONIA'], ['NORMAL', 'PNEUMONIA'])

In [5]:
val_set = val_set_default.concatenate(val_set_extra)

In [11]:
tf.random.set_seed(42)

cnn_1 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_1.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = 'accuracy'
)

history_1 = cnn_1.fit(train_set, epochs = 5, validation_data = val_set)

Epoch 1/5
155/155 [==============================] - 40s 168ms/step - loss: 0.6665 - accuracy: 0.8620 - val_loss: 0.1261 - val_accuracy: 0.9457
Epoch 2/5
155/155 [==============================] - 27s 163ms/step - loss: 0.0946 - accuracy: 0.9645 - val_loss: 0.0965 - val_accuracy: 0.9674
Epoch 3/5
155/155 [==============================] - 26s 163ms/step - loss: 0.0634 - accuracy: 0.9800 - val_loss: 0.0658 - val_accuracy: 0.9710
Epoch 4/5
155/155 [==============================] - 27s 164ms/step - loss: 0.0437 - accuracy: 0.9851 - val_loss: 0.0605 - val_accuracy: 0.9746
Epoch 5/5
155/155 [==============================] - 29s 178ms/step - loss: 0.0334 - accuracy: 0.9901 - val_loss: 0.0611 - val_accuracy: 0.9710


In [12]:
cnn_1.evaluate(test_set)

20/20 [==============================] - 3s 86ms/step - loss: 0.9549 - accuracy: 0.7580


[0.9549060463905334, 0.7580128312110901]

In [13]:
tf.random.set_seed(42)

cnn_2 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1000, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_2.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = 'accuracy'
)

history_2 = cnn_2.fit(train_set, epochs = 5, validation_data = val_set)

Epoch 1/5
155/155 [==============================] - 34s 204ms/step - loss: 3.8859 - accuracy: 0.8737 - val_loss: 0.1061 - val_accuracy: 0.9638
Epoch 2/5
155/155 [==============================] - 34s 208ms/step - loss: 0.0670 - accuracy: 0.9754 - val_loss: 0.0751 - val_accuracy: 0.9783
Epoch 3/5
155/155 [==============================] - 33s 206ms/step - loss: 0.0292 - accuracy: 0.9897 - val_loss: 0.0521 - val_accuracy: 0.9891
Epoch 4/5
155/155 [==============================] - 33s 205ms/step - loss: 0.0125 - accuracy: 0.9972 - val_loss: 0.0728 - val_accuracy: 0.9819
Epoch 5/5
155/155 [==============================] - 33s 205ms/step - loss: 0.0091 - accuracy: 0.9972 - val_loss: 0.0940 - val_accuracy: 0.9746


In [14]:
cnn_2.evaluate(test_set)

20/20 [==============================] - 3s 93ms/step - loss: 1.1134 - accuracy: 0.7981


[1.1134395599365234, 0.7980769276618958]

In [6]:
tf.random.set_seed(42)

cnn_3 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip(),
    layers.RandomRotation(0.3),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1000, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_3.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = 'accuracy'
)

history_3 = cnn_3.fit(train_set, epochs = 5, validation_data = val_set)

Epoch 1/5
155/155 [==============================] - 40s 222ms/step - loss: 2.4818 - accuracy: 0.7928 - val_loss: 0.2769 - val_accuracy: 0.8877
Epoch 2/5
155/155 [==============================] - 33s 205ms/step - loss: 0.2896 - accuracy: 0.8810 - val_loss: 0.2135 - val_accuracy: 0.9275
Epoch 3/5
155/155 [==============================] - 34s 212ms/step - loss: 0.2662 - accuracy: 0.8840 - val_loss: 0.2210 - val_accuracy: 0.9058
Epoch 4/5
155/155 [==============================] - 33s 206ms/step - loss: 0.2436 - accuracy: 0.9031 - val_loss: 0.2279 - val_accuracy: 0.9094
Epoch 5/5
155/155 [==============================] - 33s 206ms/step - loss: 0.2450 - accuracy: 0.8969 - val_loss: 0.2039 - val_accuracy: 0.9167


In [7]:
cnn_3.evaluate(test_set)

20/20 [==============================] - 3s 89ms/step - loss: 0.4811 - accuracy: 0.8269


[0.48112282156944275, 0.8269230723381042]

In [8]:
tf.random.set_seed(42)

cnn_4 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip(),
    layers.RandomRotation(0.3),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1000, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_4.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = 'accuracy'
)

history_4 = cnn_4.fit(train_set, epochs = 10, validation_data = val_set)

Epoch 1/10
155/155 [==============================] - 34s 205ms/step - loss: 3.0841 - accuracy: 0.7631 - val_loss: 0.3901 - val_accuracy: 0.8007
Epoch 2/10
155/155 [==============================] - 33s 207ms/step - loss: 0.3161 - accuracy: 0.8579 - val_loss: 0.4292 - val_accuracy: 0.8043
Epoch 3/10
155/155 [==============================] - 33s 206ms/step - loss: 0.2866 - accuracy: 0.8773 - val_loss: 0.2258 - val_accuracy: 0.9275
Epoch 4/10
155/155 [==============================] - 33s 208ms/step - loss: 0.2598 - accuracy: 0.8918 - val_loss: 0.2294 - val_accuracy: 0.9022
Epoch 5/10
155/155 [==============================] - 34s 208ms/step - loss: 0.2576 - accuracy: 0.8921 - val_loss: 0.2911 - val_accuracy: 0.8514
Epoch 6/10
155/155 [==============================] - 33s 206ms/step - loss: 0.2405 - accuracy: 0.8971 - val_loss: 0.2194 - val_accuracy: 0.9130
Epoch 7/10
155/155 [==============================] - 33s 207ms/step - loss: 0.2353 - accuracy: 0.9005 - val_loss: 0.2308 - val_ac

In [9]:
cnn_4.evaluate(test_set)

20/20 [==============================] - 3s 86ms/step - loss: 0.4466 - accuracy: 0.8558


[0.44661760330200195, 0.8557692170143127]

In [7]:
tf.random.set_seed(42)

cnn_5 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip(),
    layers.RandomRotation(0.3),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1000, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_5.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = 'accuracy'
)

history_5 = cnn_5.fit(train_set, epochs = 20, validation_data = val_set)

Epoch 1/20
155/155 [==============================] - 40s 210ms/step - loss: 2.9484 - accuracy: 0.7589 - val_loss: 0.2996 - val_accuracy: 0.8804
Epoch 2/20
155/155 [==============================] - 34s 209ms/step - loss: 0.3215 - accuracy: 0.8600 - val_loss: 0.3203 - val_accuracy: 0.8659
Epoch 3/20
155/155 [==============================] - 33s 205ms/step - loss: 0.2914 - accuracy: 0.8769 - val_loss: 0.2635 - val_accuracy: 0.8659
Epoch 4/20
155/155 [==============================] - 33s 205ms/step - loss: 0.2532 - accuracy: 0.8961 - val_loss: 0.2453 - val_accuracy: 0.9022
Epoch 5/20
155/155 [==============================] - 33s 207ms/step - loss: 0.2402 - accuracy: 0.8997 - val_loss: 0.2840 - val_accuracy: 0.8804
Epoch 6/20
155/155 [==============================] - 33s 207ms/step - loss: 0.2592 - accuracy: 0.8898 - val_loss: 0.2131 - val_accuracy: 0.9167
Epoch 7/20
155/155 [==============================] - 34s 209ms/step - loss: 0.2325 - accuracy: 0.9064 - val_loss: 0.2101 - val_ac

In [8]:
cnn_5.evaluate(test_set)

20/20 [==============================] - 3s 88ms/step - loss: 0.4313 - accuracy: 0.8558


[0.4313231110572815, 0.8557692170143127]

In [6]:
tf.random.set_seed(42)

cnn_6 = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip(),
    layers.RandomRotation(0.3),
    layers.Conv2D(filters = 32, kernel_size = 3, input_shape = (256, 256, 3), activation = 'relu'),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(1000, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'sigmoid')
])

cnn_6.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.SGD(),
    metrics = 'accuracy'
)

history_6 = cnn_6.fit(train_set, epochs = 10, validation_data = val_set)

Epoch 1/10
155/155 [==============================] - 35s 190ms/step - loss: 0.5514 - accuracy: 0.7496 - val_loss: 0.6713 - val_accuracy: 0.7174
Epoch 2/10
155/155 [==============================] - 31s 189ms/step - loss: 0.4049 - accuracy: 0.8200 - val_loss: 0.4499 - val_accuracy: 0.8043
Epoch 3/10
155/155 [==============================] - 30s 188ms/step - loss: 0.3199 - accuracy: 0.8678 - val_loss: 0.3185 - val_accuracy: 0.8768
Epoch 4/10
155/155 [==============================] - 31s 194ms/step - loss: 0.2985 - accuracy: 0.8723 - val_loss: 0.3285 - val_accuracy: 0.8587
Epoch 5/10
155/155 [==============================] - 30s 187ms/step - loss: 0.2788 - accuracy: 0.8814 - val_loss: 0.3547 - val_accuracy: 0.8188
Epoch 6/10
155/155 [==============================] - 30s 188ms/step - loss: 0.2733 - accuracy: 0.8814 - val_loss: 0.3052 - val_accuracy: 0.8804
Epoch 7/10
155/155 [==============================] - 30s 187ms/step - loss: 0.2736 - accuracy: 0.8876 - val_loss: 0.3043 - val_ac

In [7]:
cnn_6.evaluate(test_set)

20/20 [==============================] - 3s 90ms/step - loss: 0.4330 - accuracy: 0.8333


[0.4329521059989929, 0.8333333134651184]